In [1]:
%run ItemModel.ipynb
%run APIService.ipynb

In [2]:
import inspect, re
import pandas as pd
import csv
from pathlib import Path
import shutil

# Class DataCrawler

In [3]:
class DataCrawler(apiServiceDelegate):
    max_page = -1
    page = 1
    start_page = 1
    end_page = 2
    query = "environment concept art"
    source = 'artstation'
    per_page = '75'
    items_json = []
    items_img_json = []
    all_items = [] # List ARTSTATIONItem
#     all_links = []
    itemModel = ARTSTATIONItem()
    noneCategory = True
    APIService = APIService(source, query, per_page)
    i = 0
    
    @property
    def fileName(self):
        return f'ArtStationItem_page{self.page}.tsv'
    
    def __init__(self, source = 'artstation', query = 'environment concept art', per_page = '75', max_page = -1):
        self.source = source
        self.query = query
        self.per_page = per_page
        self.max_page = max_page
        self.APIService = APIService(self.source, self.query, self.per_page)
        self.APIService.delegate = self
        pass
    def setPage(self, page):
        self.page = page
        pass
    
    def pipeline_on_page(self, start_page = 1, end_page = 200):
        self.start_page = start_page
        pass
    
    def crawlData(self, start_page = 1, end_page = 200):
        while True:
            try:
                check = False
                if start_page <= self.page and self.page < end_page:
                    self.APIService.setPage(self.page)
                    check = self.APIService.get()
                if check == False: break
                elif self.max_page != -1 and self.page == (self.max_page + 1): break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def crawlImage(self):
        if len(self.all_items) != 0:
            for j in range(len(self.all_items)):
                try:
                    self.APIService.img_hash_id = self.all_items[j].hash_id
                    check = self.APIService.get_img()
                    if check == False: break
                except Exception as e:
#                     self.parseData('img')
                    print(f'Error Crawl Image: {e}')
                    break
                self.i += 1
        else:
            print('len(all_items) == 0')
        pass
    
    def parseData(self):
        block = """/\*?|:"<>"""
        i = 0
        f = open('download.json', encoding="utf8")
        f
        # returns JSON object as
        # a dictionary
        self.items_json = json.load(f)
        self.all_items.clear()
        for item in self.items_json:
            itemModel = ARTSTATIONItem()
            itemModel.link = item['photoModalImageSrc']
            if(item['photoModalTitle'] == None):
                itemModel.name = str(i) + '_None'
#                 i += 2
            else:
                name = re.sub("""[/\\\\*?|:"<>]""", '', item['photoModalTitle'])
                itemModel.name = str(i) + '_' + name
            i+=1
            self.all_items.append(itemModel)
            del itemModel
            itemModel = None
        pass
    def parseImage(self):
        print('parseImage ', self.i, ': ', len(self.items_img_json), ' url')
        itemModel = []
#         itemModel.append(['.'])
        for item in self.items_img_json:
# #             print(item['image_url'])
            itemModel.append(item['image_url'])
# #         print('---',itemModel.link)
                
#         self.all_items[self.i].link = ['1']
        self.all_items[self.i].link = itemModel
        del itemModel
        itemModel = None
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'hash_id\turl\n')
 
        
        for item in self.all_items:
                file.write(f'{item.hash_id}\t{item.link}\n')
    
    def downloadImage(self):
        p = Path('E:/study/crawl-face-image/masked-face-image')
        for item in self.all_items[852:]:
            r = requests.get(item.link, stream=True)
            if r.status_code == 200:
                with open(f"{p}/{item.name}.jpg", 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f)
                
        pass
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata['data']))
        print(f'get! {self.page}')
        
        if 'data' in json_pydata:
            self.items_json.extend(json_pydata['data'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.page += 1
        time.sleep(3)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    def updateImageWhenSuccess(self, json_pydata):
#         print(f'Update link at item {self.i}')
        self.items_img_json.clear()
        if 'assets' in json_pydata:
            self.items_img_json = json_pydata['assets']
            self.parseImage()
        else:
#             self.parseData('img')
            print('get All Image Link!')
            return False
#         print(f' Done Update link at item {self.i}')
        time.sleep(0.3)
        return True
        pass
    def getImageError(self, error):
        print(f'Delegate: getImageError: {error}')
#         self.parseData('img')
        return False
        pass    
    pass

# Crawling Data

In [4]:
crawler = DataCrawler(per_page = 75)

In [5]:
crawler.parseData()

In [6]:
crawler.downloadImage()

In [5]:
crawler.crawlData()


75
get! 1
75
get! 2
75
get! 3
75
get! 4
75
get! 5
75
get! 6
75
get! 7
75
get! 8
75
get! 9
75
get! 10
75
get! 11
75
get! 12
75
get! 13
75
get! 14
75
get! 15
75
get! 16
75
get! 17
75
get! 18
75
get! 19
75
get! 20
75
get! 21
75
get! 22
75
get! 23
75
get! 24
75
get! 25
75
get! 26
75
get! 27
75
get! 28
75
get! 29
75
get! 30
75
get! 31
75
get! 32
75
get! 33
75
get! 34
75
get! 35
75
get! 36
75
get! 37
75
get! 38
75
get! 39
75
get! 40
75
get! 41
75
get! 42
75
get! 43
75
get! 44
75
get! 45
75
get! 46
75
get! 47
75
get! 48
75
get! 49
75
get! 50
75
get! 51
75
get! 52
75
get! 53
75
get! 54
75
get! 55
75
get! 56
75
get! 57
75
get! 58
75
get! 59
75
get! 60
75
get! 61
75
get! 62
75
get! 63
75
get! 64
75
get! 65
75
get! 66
75
get! 67
75
get! 68
75
get! 69
75
get! 70
75
get! 71
75
get! 72
75
get! 73
75
get! 74
75
get! 75
75
get! 76
75
get! 77
75
get! 78
75
get! 79
75
get! 80
75
get! 81
75
get! 82
75
get! 83
75
get! 84
75
get! 85
75
get! 86
75
get! 87
75
get! 88
75
get! 89
75
get! 90
75
get! 91
75
get! 

In [6]:
crawler.parseData()

In [7]:
crawler.crawlImage()

parseImage  0 :  14  url
parseImage  1 :  4  url
parseImage  2 :  1  url
parseImage  3 :  1  url
parseImage  4 :  4  url
parseImage  5 :  5  url
parseImage  6 :  1  url
parseImage  7 :  6  url
parseImage  8 :  1  url
parseImage  9 :  4  url
parseImage  10 :  6  url
parseImage  11 :  2  url
parseImage  12 :  6  url
parseImage  13 :  4  url
parseImage  14 :  1  url
parseImage  15 :  1  url
parseImage  16 :  6  url
parseImage  17 :  2  url
parseImage  18 :  1  url
parseImage  19 :  4  url
parseImage  20 :  4  url
parseImage  21 :  3  url
parseImage  22 :  4  url
parseImage  23 :  4  url
parseImage  24 :  9  url
parseImage  25 :  2  url
parseImage  26 :  2  url
parseImage  27 :  2  url
parseImage  28 :  1  url
parseImage  29 :  2  url
parseImage  30 :  1  url
parseImage  31 :  2  url
parseImage  32 :  2  url
parseImage  33 :  7  url
parseImage  34 :  7  url
parseImage  35 :  5  url
parseImage  36 :  2  url
parseImage  37 :  2  url
parseImage  38 :  3  url
parseImage  39 :  4  url
parseImag

In [8]:
crawler.downloadImage()

ProtocolError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [ ]:
# crawler.items_img_json

In [ ]:
crawler.all_items[9].link

In [ ]:
# Check last request
# print(json.dumps(crawler.APIService.json_pydata['data'], indent = 3, sort_keys=True))

In [ ]:
# Check feature value
crawler.all_items[0].hash_id

In [ ]:
# Test ParseData
print(crawler.all_items[0].hash_id)
print(crawler.all_items[9].hash_id)
# print(crawler.all_items[10].hash_id)
# print(crawler.all_items[19].hash_id)

In [9]:
crawler.saveFile('art.csv')

In [10]:
# Test saveFile
items_df = pd.read_csv('art.csv', sep='\t')
pd.set_option('max_colwidth', None)

items_df



,hash_id,url
0,4boNZ4,"['https://cdna.artstation.com/p/assets/covers/images/022/118/768/large/christian-dimitrov-02.jpg?1574182578', 'https://cdna.artstation.com/p/assets/images/images/028/839/734/large/christian-dimitrov-01-legendary-2500.jpg?1595675385', 'https://cdna.artstation.com/p/assets/images/images/028/839/740/large/christian-dimitrov-02-legendary-2500.jpg?1595675393', 'https://cdna.artstation.com/p/assets/images/images/028/839/744/large/christian-dimitrov-03-legendary-2500.jpg?1595675402', 'https://cdnb.artstation.com/p/assets/images/images/028/839/749/large/christian-dimitrov-04-legendary-2500.jpg?1595675410', 'https://cdnb.artstation.com/p/assets/images/images/028/839/785/large/christian-dimitrov-05-legendary-2500.jpg?1595675484', 'https://cdna.artstation.com/p/assets/images/images/028/839/788/large/christian-dimitrov-06-legendary-2500.jpg?1595675490', 'https://cdnb.artstation.com/p/assets/images/images/028/839/789/large/christian-dimitrov-07-legendary-2500.jpg?1595675496', 'https://cdnb.artstation.com/p/assets/images/images/028/839/791/large/christian-dimitrov-08-legendary-2500.jpg?1595675502', 'https://cdnb.artstation.com/p/assets/images/images/036/780/985/large/christian-dimitrov-09-legendary-2500.jpg?1618588862', 'https://cdna.artstation.com/p/assets/images/images/028/839/812/large/christian-dimitrov-10-legendary-2500.jpg?1595675632', 'https://cdnb.artstation.com/p/assets/images/images/028/839/815/large/christian-dimitrov-11-legendary-2500.jpg?1595675637', 'https://cdna.artstation.com/p/assets/images/images/028/839/818/large/christian-dimitrov-12-legendary-2500.jpg?1595675644', 'https://cdna.artstation.com/p/assets/images/images/036/780/988/large/christian-dimitrov-13-legendary-2500.jpg?1618588874']"
1,KrJKLx,"['https://cdna.artstation.com/p/assets/images/images/033/530/544/large/sergey-samsonov-1-concept-environment-cargarbage-visha-published.jpg?1609869861', 'https://cdnb.artstation.com/p/assets/images/images/033/529/927/large/sergey-samsonov-2-concept-environment-cargarbage-office-sails-ublished.jpg?1609868704', 'https://cdnb.artstation.com/p/assets/images/images/033/529/931/large/sergey-samsonov-3-concept-environment-cargarbage-old-elevator-published.jpg?1609868710', 'https://cdnb.artstation.com/p/assets/images/images/033/529/937/large/sergey-samsonov-4-concept-environment-cargarbage-published.jpg?1609868718']"
2,DwaWy,['https://cdna.artstation.com/p/assets/images/images/008/897/794/large/branislav-perkovic-13.jpg?1515977576']
3,xoNEX,['https://cdna.artstation.com/p/assets/images/images/004/247/572/large/michel-donze-forestsconcept-upper.jpg?1481706939']
4,Eg1Aq,"['https://cdnb.artstation.com/p/assets/images/images/004/900/545/large/francis-tneh-pitch-pack-cyberpunk-downtown-final.jpg?1487085945', 'https://cdna.artstation.com/p/assets/images/images/004/900/544/large/francis-tneh-ds-env-01.jpg?1487085876', 'https://cdnb.artstation.com/p/assets/images/images/004/900/547/large/francis-tneh-pitch-pack-darkelf-palace-final.jpg?1487085884', 'https://cdna.artstation.com/p/assets/images/images/004/900/548/large/francis-tneh-pitch-pack-elf-palace-final.jpg?1487085889']"
...,...,...
9970,YlR53,[]
9971,1DAw2,[]
9972,oWZXk,[]
9973,Govnz,[]


In [ ]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [ ]:
del items_df